

```
# Setup
```

In [ ]:
!nvidia-smi # runtime T4 GPU ашиглана

# --- Гол libraries суулгах
!pip -q install -U "transformers[torch]" datasets accelerate peft sentencepiece evaluate rouge_score bitsandbytes wandb

# --- Import test хийх, version -г хэвлэх
import torch, transformers, datasets, peft, evaluate
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("Datasets:", datasets.__version__)
print("PEFT:", peft.__version__)



Mon Oct 13 08:02:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# --- Hugginface -руу нэтврэх
from huggingface_hub import login
login()

# preprocess хийхийн өмнө schema -г бататгах
Load & inspect dataset

In [ ]:
from datasets import load_dataset

# 1) Load all splits
ds = load_dataset("amaraaa/mn_translated_cnn")
print(ds)

# 2) Train split -ээс хэдэн мөрүүдийг шалгая
for i in range(2):
  row = ds["train"][i]
  print(f"nRow {i} keys:", row.keys())
  print("id:", row["id"])
  print("article snippet:", (row["article"] or "")[:200].replace("\n"," "))
  print("highlights (type/len):", type(row["highlights"]).__name__, len(row["highlights"]))
  print("first highlight:", row["highlights"][0] if row["highlights"] else "-")

# 3) Sanity checks - Алдаанаас сэргийлж шалгалт хийх (data sanity checks)
def summarize_split(name):
  d = ds[name]
  # Хоосон article байгаа эсэхийг шалгах
  empty_articles = sum(1 for r in d if not r["article"] or not r["article"].strip())
  # Хоосон highlights байгаа эсэхийг шалгах
  empty_highlights = sum(1 for r in d if not r"highlights" or not r["highlights".strip()])
  return {
      "rows": len(d),
      "empty_articles": empty_articles,
      "empty_highlights": empty_highlights,
  }

print("\nSanity:")
for split in ds.keys():
  print(split, summarize_split(split))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/602 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.45M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/86.0k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/60.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/369 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/20 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/17 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'article', 'highlights'],
        num_rows: 369
    })
    validation: Dataset({
        features: ['id', 'article', 'highlights'],
        num_rows: 20
    })
    test: Dataset({
        features: ['id', 'article', 'highlights'],
        num_rows: 17
    })
})
nRow 0 keys: dict_keys(['id', 'article', 'highlights'])
id: 0000800d9058217f6509d7e63ad475e2de0da611
article snippet: . Ричард Спиллетт. Машинд цэнэглэж байх үед дэлбэрсэн электрон тамхи нь бүтээгдэхүүний аюулгүй байдлын талаар шинэ айдас төрүүлэв. 54 настай асран хамгаалагч Ким Тейлор машинаас гал болон өтгөн хар ут
highlights (type/len): list 5
first highlight: 54 настай Ким Тейлор төхөөрөмжөө Ford Mondeo машины арын суудалд залгаад орхижээ.
nRow 1 keys: dict_keys(['id', 'article', 'highlights'])
id: 0000bf554ca24b0c72178403b54c0cca62d9faf8
article snippet: . Алекс Уорд. Баян хөрөнгө оруулагчдаас олон сая фунт стерлинг залилсан хотын худалдаачин өчигдөр нэг фунт стерл

# Build preprocessing - (model training -д зориулж өгөгдлийг бэлдэх)
* mT5 нь single target string хүлээж авдаг
* highlights -д мөр бүрт string list байгаа

1) Иймээс list -ийн агуулгыг newline -тай нэг text болгоно
2) inputs/targets -ийг tokenize хийнэ (consistent max lengths)

In [ ]:
from datasets import DatasetDict
from transformers import AutoTokenizer

# --- Тохируулга хийх
model_name = "google/mt5-small"
task_prefix = "summarize: " # t5 model -д ямар таск хийхийг prefix зааж өгнө
max_source_length = 768
max_target_length = 128

# 1) Highlight -уудын list -ийг single string болгож join хийх
def join_highlights(example): # (example -> single record)
  bullets = example.get("highlights") or []
  # keep only non-empty strings and strip whitespace
  bullets = [str(x).strip() for x in bullets if isinstance(x, str) and str(x).strip()]
  example["highlights_str"] = "\n".join(bullets)
  # print(example)
  return example

# Функцийг split бүрийн record дээр ажилуулна
ds = ds.map(join_highlights, desc="Joining highlight bullets")

# 2) Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 3) Tokenize inputs & targets - (article, highlights)
def preprocess_batch(batch):
  # input text + task hint
  inputs = [task_prefix + (article or "") for article in batch["article"]]
  targets = batch["highlights_str"]

  # tokenize sources
  model_inputs = tokenizer(
      inputs,
      max_length=max_source_length,
      truncation=True,
  )

  # tokenize targets (labels)
  labels = tokenizer(
      text_target=targets,
      max_length=max_target_length,
      truncation=True,
  )

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

# cols_to_remove = list(set(ds["train"].column_names) - {"id"})
# Remove original columns and the joined highlights string, including 'id'
cols_to_remove = ["id", "article", "highlights", "highlights_str"]

tokenized_ds = ds.map(
    preprocess_batch,
    batched=True,
    remove_columns=cols_to_remove,
    desc="Tokenizing"
)

print(tokenized_ds)
print("Example ids:", tokenized_ds["train"][0]["input_ids"][:20], "...")
print("Example labels:", tokenized_ds["train"][0]["labels"][:20], "...")

# --- Tokenized хослолыг шалгах
sample = tokenized_ds["train"][0]
print(tokenizer.decode(sample["input_ids"][:128], skip_special_tokens=True))

# Safe label preview: (replace -100 with pad id before decoding)
safe_labels = [(tid if tid != -100 else tokenizer.pad_token_id) for tid in sample["labels"]]
print(tokenizer.decode(safe_labels[:64], skip_special_tokens=True))



Joining highlight bullets:   0%|          | 0/369 [00:00<?, ? examples/s]

Joining highlight bullets:   0%|          | 0/20 [00:00<?, ? examples/s]

Joining highlight bullets:   0%|          | 0/17 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

...

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Tokenizing:   0%|          | 0/369 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/20 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/17 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 369
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 20
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 17
    })
})
Example ids: [196098, 10701, 267, 259, 260, 1936, 176023, 902, 688, 94817, 237497, 260, 259, 96430, 902, 259, 68991, 129424, 990, 17157] ...
Example labels: [7013, 310, 85482, 1035, 1104, 1454, 1011, 104221, 16661, 605, 72599, 17159, 31915, 10241, 61397, 268, 259, 50835, 259, 20903] ...
summarize: . Ричард Спиллетт. Машинд цэнэглэж байх үед дэлбэрсэн электрон тамхи нь бүтээгдэхүүний аюулгүй байдлын талаар шинэ айдас төрүүлэв. 54 настай асран хамгаалагч Ким Тейлор машинаас гал болон өтгөн хар утаа гарч байгааг анзаарахаасаа өмнө аравхан минутын турш төхөөрөмжийг цэнэглэж орхижээ. Тухайн үед дотор нь хэн ч байгаагүй ч арын суудал бүрэн 
54 на

# Load mT5-small & attach LoRa adapters
LoRa гэх мэт PEFT method -оор model сургахдаа бэлдэх (peft configuration)

LoRa -> суурь model -ийг бүхэлд нь биш зөвхөн тодорхой цөөн weight -үүдийг сургана.
*   Resrource бага шаардана
*   Илүү efficient байх боломжтой
*   Model -ийн цөөн тооны parameter -ийг үр дүнтэй fine-tune хийнэ.
*   Computational болон storage cost багасгана.


In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model, TaskType

#1) Load base model
base_model_name = "google/mt5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)

#2) (recommended for memory) - enable gradient checkpointing later during training
model.config.use_cache = False
# model.config.decoder_start_token_id = tokenizer.pad_token_id

#3) LoRa config тохируулах
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=16, # rank (capacity of adapters)
    lora_alpha=32, #scaling
    lora_dropout=0.1, # regularization
    target_modules=["q", "v"], # T5 attention projections commonly adapted
)

#4) Wrap model with LoRa - (mt5 -д loRa technique -ийг apply хийнэ)
model = get_peft_model(model, peft_config)

# Enable input gradients for gradient checkpointing
model.enable_input_require_grads()

# Move the model to the GPU
model.to("cuda")


#5) Quick sanity: хэчнээн params сургахаа харах
model.print_trainable_parameters()

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

trainable params: 688,128 || all params: 300,864,896 || trainable%: 0.2287


In [ ]:
# 2) Sentinel blocklist үүсгэх (T5 extra_id 0..99) -> for generation
vsize = tokenizer.vocab_size
extra_ids = list(range(vsize - 100, vsize))
bad_words_ids = [[i] for i in extra_ids]
print(f"Blocking {len(bad_words_ids)} sentinel IDs: {extra_ids[0]}..{extra_ids[-1]}")

# Trainer тохируулга (metrics, collator, hyperparams)
Rouge metrics, batching/padding, hyperparameters тохируулах


In [ ]:
import numpy as np
import evaluate
from transformers import(
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    EarlyStoppingCallback,
)

#1) Metric: ROUGE
rouge = evaluate.load("rouge")

def postprocess_text(preds, labels):
  # strip хийгээд sentence хоороod newline нэмэх
  preds = [p.strip() for p in preds]
  labels = [l.strip() for l in labels]
  return preds, labels

def compute_metrics(eval_pred):
  preds, labels = eval_pred

  # Some trainers return (sequences)
  if isinstance(preds, tuple):
      preds = preds[0] #take token IDs matrix

  # IMPORTANT: decode хийхээс өмнө ignore index (-100) -г real pad token ID -аар солих
  #preds
  if isinstance(preds, np.ndarray):
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
  else:
    # fallback in case preds is a list
    preds = [[(tok if tok != -100 else tokenizer.pad_token_id) for tok in seq] for seq in preds]
  #labels
  if isinstance(labels, np.ndarray):
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  else:
    # fallback in case labels is a list
    labels = [[(tok if tok != -100 else tokenizer.pad_token_id) for tok in seq] for seq in labels]

  # print("Debugging: (computer_metrics) ")
  # print("Type of preds:", type(preds))
  # print("Preds -> first 50 values", preds[:50])
  # print("---------------------------------")

  # decode predictions
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  print(decoded_preds)

  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # light cleanup for ROUGE
  decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

  # Compute ROUGE
  result = rouge.compute(
      predictions=decoded_preds,
      references=decoded_labels,
      use_stemmer=True,
  )

  # # add a smiple generation length metric
  # gen_lens = [np.count_nonzero(p != tokenizer.pad_token_id) for p in preds]
  # result["gen_len"] = float(np.mean(gen_lens))
  # # focus metric alias
  # result["rougeLsum"] = result.get("rougeLsum", result.get("rougeL", 0.0))
  # return {k: round(v * 100, 4) if k.startswith("rouge") else round(v, 2) for k, v in result.items()}

  return {k: round(v * 100, 4) for k, v in result.items()}

#2) dynamic padding & label masking (-100)
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8 if torch.cuda.is_available() else None,
)

#3) Enable memory savers
model.gradient_checkpointing_enable() # reduces VRAM

#4) Training arguments (tuned for small data + LoRA)
training_args = Seq2SeqTrainingArguments(
    output_dir="mt5_mncnn_lora",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=5e-4,
    num_train_epochs=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    predict_with_generate=True,
    generation_max_length=128,
    generation_num_beams=4,
    warmup_ratio=0.03,
    # lr_scheduler_type="cosine",
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="eval_rougeLsum",
    greater_is_better=True,
    fp16=False,  #AMP on Colab GPUs
    # label_smoothing_factor=0.1,
    # max_grad_norm=0.5,
    # remove_unused_columns=True,
    report_to="none",

    # Decode hygiene
    # no_repeat_ngram_size=3,
    # length_penalty=1.0,
    # early_stopping=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Add early stopping (prevents overfitting on 369 rows)
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))

print("Ready to train.")
print("Train rows:", len(tokenized_ds["train"]), "| Val rows:", len(tokenized_ds["validation"]))

Ready to train.
Train rows: 369 | Val rows: 20


/tmp/ipython-input-2612925576.py:113: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


# Sanity check before full training

In [ ]:
# Small slice of eval to test metrics & decoding path
small_eval = tokenized_ds["validation"].select(range(20))
eval_result = trainer.evaluate(eval_dataset=small_eval)

print(eval_result)


['<extra_id_0>.com-д.mn', '<extra_id_0>.', '<extra_id_0>сан.', '<extra_id_0> байдаг.', '<extra_id_0>.', '<extra_id_0> их өвдөж байсан.', '<extra_id_0>.', '<extra_id_0> байдаг юм.', '<extra_id_0>.com-д хэрэглэдэг хуванцар сав.', '<extra_id_0>.com', '<extra_id_0> байна.', '<extra_id_0>.', '<extra_id_0> .', '<extra_id_0>.com', '<extra_id_0> байдаг. (CNN)', '<extra_id_0> болон бусад гурван хүн амиа алджээ.', '<extra_id_0>.', '<extra_id_0> байдаг.', '<extra_id_0> байдаг. Soul Penthouse', '<extra_id_0>.com']
{'eval_loss': 10.768037796020508, 'eval_model_preparation_time': 0.0068, 'eval_rouge1': 1.3333, 'eval_rouge2': 0.7692, 'eval_rougeL': 1.3333, 'eval_rougeLsum': 1.3333, 'eval_runtime': 4.5613, 'eval_samples_per_second': 4.385, 'eval_steps_per_second': 1.096}


In [ ]:
# Small slice of eval to test metrics & decoding path
small_eval = tokenized_ds["train"].select(range(20))
eval_result = trainer.evaluate(eval_dataset=small_eval)

print(eval_result)


['<extra_id_0> .', '<extra_id_0>.', '<extra_id_0>. . . . . . . . . . . .', '<extra_id_0> .', '<extra_id_0> байна.', '<extra_id_0> . Скармардо . Скармардо . Скармардо . Скармардо . Скармардо . Скармардо . Скармардо . Скармардо . Скармардо . Скармардо . Скармардо .', '<extra_id_0> байдаг.', '<extra_id_0>.', '<extra_id_0> байдаг.', '<extra_id_0>.com-д.mn', '<extra_id_0> байдаг. . 21:22 EST, 3 9-р сарын 2012 . 21:22 EST, 3 9-р сарын 2012 .', '<extra_id_0>.com-д.mx', '<extra_id_0> .', '<extra_id_0>. . . . . . . . . . . .', '<extra_id_0>гүй.', '<extra_id_0> байдаг юм.', '<extra_id_0>гүй байна. (CNN)', '<extra_id_0> байдаг.', '<extra_id_0> менежер юм.', '<extra_id_0>.com-д.']
{'eval_loss': 14.388493537902832, 'eval_model_preparation_time': 0.0068, 'eval_rouge1': 0.0, 'eval_rouge2': 0.0, 'eval_rougeL': 0.0, 'eval_rougeLsum': 0.0, 'eval_runtime': 4.9867, 'eval_samples_per_second': 4.011, 'eval_steps_per_second': 1.003}


# Training эхлүүлэх - (LoRa on mT5-small)

In [ ]:
# # ──────────────────────────────────────────────────────────────
# # 8) Evaluate → Train → Evaluate
# # ──────────────────────────────────────────────────────────────
print("Eval (before training):")
metrics_before = trainer.evaluate()
print(metrics_before)

# Training
train_result = trainer.train()

# Save the best checkpoint (load_best_model_at_the_end=True)
trainer.save_model("mt5_mncnn_lora/best") # saves LoRa adapter weights + config
tokenizer.save_pretrained("mt5_mncnn_lora/best")

# see a short summary of training
print(train_result)

# Eval after training
print("Eval (before training):")
metrics_after = trainer.evaluate()
print(metrics_after)

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,4.816060,0.000000,0.000000,0.000000,0.000000
2,No log,3.495636,0.000000,0.000000,0.000000,0.000000
3,9.425200,2.796236,2.666700,1.538500,2.666700,2.666700
4,9.425200,2.808631,1.538500,0.909100,1.538500,1.538500
5,5.293500,2.867482,5.481200,3.088200,5.105300,5.481200
6,5.293500,2.879550,12.500900,6.928700,12.481200,12.567300
7,4.504400,2.889411,7.166700,5.000000,7.021900,7.105300
8,4.504400,2.893785,7.111100,5.000000,6.928100,7.019600


['<extra_id_0> нь ялалт байгуулсан гэж үзсэн байна. Тэд түүний түүхэн алдааг ялалт байгуулсан нь ялалт байгуулсан гэж үзэж байна. Тэд түүнийг нь ялалт байгуулсан нь ялалт байгуулсан нь ялалт байгуулсан гэж үзсэн байна. Тэд нь түүнийг ялалт байгуулсан цаг нь ялалт байгуулсан нь ялалт байгуулсан бол энэ нь ялалт байгуулсан бол энэ нь ялалт байгуулсан түүнийг ', '<extra_id_0> нь түүнийг өөрийн амьд авч, түүнийг нь ашигласан байна. Гэвч түүнийг нь түүнийг нь өөрийн амьд авч, түүнийг өөрийн амьд авч, түүнийг өөрийн амьд авч, түүнийг өөрийн амьд авч, түүнийг өөрийн амьд авч, түүнийг өөрийн амьд авч, түүнийг өөрийн амьд авч, түүнийг өөрийн амьд авч, түүнийг ', '<extra_id_0> байсан нь түүнийг гүйгчээ. Тэр нь түүнийг гүйгчээ. Тэр нь түүнийг гүйгчээ. Тэр нь түүнийг гүйгчээ. Тэрээр нь түүнийг гүйгчээ. Тэр нь түүнийг гүйгчээ. Мөн түүнийг гүйгчээ. Мөн түүнийг гүйгчээ. Мөн түүнийг түүнийг гүйгчээ. Тэрээр түүнийг энэ нь түүнийг энэ нь түүнийг', '<extra_id_0> яахав. Гэвч энэ нь дэлхий дээр үргэлжилсэн

In [ ]:
# Сургасан model -оо хадгалах
import os

os.makedirs("./trained_for_summarization_mn", exist_ok=True)
if hasattr(trainer.model, "module"):
  trainer.model.module.save_pretrained("./trained_for_summarization_mn")
else:
  trainer.model.save_pretrained("./trained_for_summarization_mn")

# Сургасан model-оо local -аас ачааллах
# from transformers import AutoModelForSeq2SeqLM

# model = (AutoModelForSeq2SeqLM
#          .from_pretrained("./trained_for_summarization_mn")
#          .to(device))

# Lock the best checkpoint, evaluate on test & inspect generations (with sentinel-blocking)

*   Best checkpoint -> хадгалах
*   Test dataset дээр evaluate хийх
*   Generations -ийг шалгах (block **sentinel tokens**)


In [ ]:
# 1) Best checkpoint -ийг хадгалах
import torch, numpy as np
from torch.utils.data import DataLoader
from transformers import AutoModelForSeq2SeqLM
from peft import PeftModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_ckpt = getattr(trainer.state, "best_model_checkpoint", None)
print("Best checkpoint:", best_ckpt)

# If None, fall back to trainer.model as-is; else reload clean
if best_ckpt:
  base_eval = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")
  model_eval = PeftModel.from_pretrained(base_eval, best_ckpt).to(device)
else:
  model_eval = trainer.model.to(device)

model_eval.eval()

# 2) Sentinel blocklist үүсгэх (T5 extra_id 0..99) -> for generation
vsize = tokenizer.vocab_size
extra_ids = list(range(vsize - 100, vsize))
bad_words_ids = [[i] for i in extra_ids]
print(f"Blocking {len(bad_words_ids)} sentinel IDs: {extra_ids[0]}..{extra_ids[-1]}")

# 3) DataLoader for test split
test_loader = DataLoader(
    tokenized_ds["test"],
    batch_size=4,
    shuffle=False,
    collate_fn=data_collator,
)

# 4) Generate with constraints & compute ROUGE on the test set
rouge = evaluate.load("rouge")
all_preds, all_refs = [], []

for batch in test_loader:
  labels = batch["labels"].numpy()
  refs = tokenizer.batch_decode(
    np.where(labels != -100, labels, tokenizer.pad_token_id),
    skip_special_tokens=True,
  )
  with torch.no_grad():
    gen = model_eval.generate(
        input_ids=batch["input_ids"].to(device),
        attention_mask=batch["attention_mask"].to(device),
        num_beams=4,
        max_new_tokens=128,
        no_repeat_ngram_size=3, #reduce loops
        repetition_penalty=1.15,
        bad_words_ids=bad_words_ids, #block sentinel tokens
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )
  preds = tokenizer.batch_decode(gen, skip_special_tokens=True)

  all_refs.extend([r.strip() for r in refs])
  all_preds.extend([p.strip() for p in preds])

test_metrics = rouge.compute(
    predictions=all_preds,
    references=all_refs,
    use_stemmer=False,
)
test_metrics = {k: round(v * 100, 2) for k, v in test_metrics.items()}
print("TEST ROUGE:", test_metrics)

# 5) Show a few qualitative examples
def show_pairs(n=5):
  for i in range(n):
    print(f"\n--- Example{i+1} ---")
    print("Pred:", all_preds[i][:400])
    print("Ref:", all_refs[i][:400])

show_pairs(5)




TEST ROUGE: {'rouge1': np.float64(13.33), 'rouge2': np.float64(7.84), 'rougeL': np.float64(13.53), 'rougeLsum': np.float64(13.73)}

--- Example1 ---
Pred: langsung: Хуан Аранго (зүүн талд) Мексикт болсон тоглолтод оролцсон. Тэрээр Венесуэлийн лигт Монтеррейд 4:3-аар хожигдсоныг дурсан санаж байсан ч гайхалтай торгуулийн цохилтыг хийсэн юм.
Ref: Хуан Аранго Хесус Завелаг хазсаны улмаас шүүгчийн шийтгэлээс мултарсан. Тэр үйл явдлын төлөө буцаан шийтгэл хүлээж магадгүй юм. Аранго өмнө нь багаа 4:3-аар буулган авахад торгуулийн цохилт хийж байсан.

--- Example2 ---
Pred: langsung. Гарднер Сити Граунд Уотфордод хожигдох үеэр Виллагийн босс томилогдсоноосоо хойш амжилттай зээлээр авах боломжтой эсэхийг харахыг хүсч байгаа.
Ref: Гари Гарднер улирлын өмнөх бэлтгэлд хамрагдахаар Астон Виллад очно гэдгээ баталжээ. 22 настай тэрээр Чемпионшип баг болох Ноттингем Форест багт зээлээр тоглосон. Тим Шервуд ирэх улирлын өмнө Гарднерыг шүүмжлэх сонирхолтой байгаа. Хэрвээ Вилла сэрэхгүй бол хагас хамгаа

In [ ]:
# print(ds)
# print(ds["train"])
# print(ds["train"][1]["highlights"])

# def square(number):
#   return number * number

# number = [1, 2, 3, 4 , 5]
# square_nums = map(square, number)
# print(list(square_nums))